In [ ]:
#!/usr/bin/env python2.7
import sys
import pdb
import argparse
import random 
import numpy as np
import copy
from Bio import SeqIO
from itertools import permutations, combinations, repeat
from StringIO import StringIO
from guppy import hpy
import time
import multiprocessing

start = time.time()
h = hpy()



def score_nt(mismatch=-4, indel=-8, match=5):
  score_pair = {}
  #store all possible mismatch and indel scores into ScoreTwoChar Hash table
  nt = ['A', 'T', 'C', 'G', '-']
  for a in nt:
    for b in nt:
      if a == b :
        if not a == '-':
          score_pair[(a, b)] = match
        else:
          score_pair[(a, b)] = 0
      else:
        if a == '-' or b == '-':
          score_pair[(a, b)] = indel
        else:
          score_pair[(a, b)] = mismatch
  score_three = {}
  for a in nt:
    for b in nt:
      for c in nt:
        if a == b and b == c and c == '-':
          continue
        else:
          score_three[(a, b, c)] = sum(score_pair[(a,b)], score_pair[(a,c)], score_pair[(b,c)]
  return score_three

score = score_nt()





def scoreTwoSeq(x, y):
  """This function returns a 2D score matrix.
  even though the name of function says this is a two sequence scoring function
  actually in this function  the score is caluclated as if calculating three sequences
  """
  global score

  indel_2 = score[('A', '-', '-')]
  new_match = score[('A', 'A', '-')]
  new_mismatch = score[('A', 'C', '-')]

  M = np.zeros((len(x) + 1, len(y) + 1))
  Path = np.empty((len(x) + 1, len(y) + 1), dtype=object)

  for i in range(1, len(y) + 1):
    M[0][i] = i * new_indel
    Path[0][i] = (0, i-1)
  for j in range(1, len(x) + 1):
    M[j][0] = j * new_indel 
    Path[j][0] = (j-1, 0)
####################### # check initialization 
  for i in range(1, len(x) + 1):
    for j in range(1, len(y) + 1):
      if x[i-1] == y[j-1]:
        M[i][j] = max(M[i-1][j-1] + new_match, M[i-1][j] + new_indel, M[i][j-1] + new_indel)
        if M[i][j] == M[i-1][j-1] + new_match:
          Path[i][j] =  (i-1, j-1)
        elif M[i][j] == M[i-1][j] + new_indel:
          Path[i][j] = (i-1, j)
        else:
          Path[i][j] = (i, j-1)
      else:
        M[i][j] = max(M[i-1][j-1] + new_mismatch, M[i-1][j] + new_indel, M[i][j-1] + new_indel)
        if M[i][j] == M[i-1][j-1] + new_mismatch:
          Path[i][j] =  (i-1, j-1)
        elif M[i][j] == M[i-1][j] + new_indel:
          Path[i][j] = (i-1, j)
        else:
          Path[i][j] = (i, j-1)
  return M, Path

def scoreThreeSeq(seq1, seq2, seq3):
  """
  This function returns the last surface of the alignment 
  between three sequences.
  """
  global score
  new_indel = score[('A', '-', '-')]
  prev, Path = scoreTwoSeq(B, C)
  
  new = np.zeros((len(B) + 1, len(C) + 1)) 
  
    for a in range(1, len(A) + 1):
    new[0, 0] = prev[0, 0] + new_indel

  # fill in the dumpy row when B == 0
    for c in range(1, len(C) + 1):
      new[0, c] = max(new[0, c-1] + new_indel, prev[0, c] + new_indel, 
          prev[0, c-1] + score[(A[a-1], C[c-1], '-')])
  # fill in the dumpy row when C == 0
    for b in range(1, len(B) + 1):
      new[b, 0] = max(new[b-1, 0] + new_indel, prev[b, 0] + new_indel, 
          prev[b-1, 0] + score[(A[a-1], B[b-1], '-')])
     
    for b in range(1, len(B) + 1):
        for c in range(1, len(C) + 1):
        #1-3 edges, 4-6 diagnoal edges, 7, cube diagnoal edge
        seven = prev[b-1, c-1] + score[(A[a-1], B[b-1], C[c-1])]
        six   = prev[b-1, c] + score[(A[a-1], B[b-1], '-')]
        five  = prev[b, c-1] + score[(A[a-1], '-', C[c-1])]
        three = prev[b, c] + score[(A[a-1], '-', '-')]
        four  = new[b-1, c-1] + score[('-', B[b-1], C[c-1])]
        two   = new[b-1, c] + score[('-', B[b-1], '-')]
        one   = new[b, c-1] + score[('-', '-', C[c-1])]
        new[b, c] =  max([one, two, three, four, five, six, seven])
    prev = copy.deepcopy(new)
  return new


def alignThreeSeq(A, B, C):
  global score

  M = np.zeros((len(A) + 1, len(B) + 1, len(C) + 1))
#  np.ndarray.fill(M[0, :, :] , -100)

  P = np.empty((len(A) + 1, len(B) + 1, len(C) + 1), dtype=object) 
# A number is assigned to P[a, b, c] to traceback.
# please take a look at the fig here 3DAlign.pdf if you don't know which corner 
# of the cube does a specific number refers to 
  ################################################
  # initialization

# initialize three dummy plane
  M[0, :, : ], P_tmp = scoreTwoSeq(B, C)
  # below magic line converts an two dimentional tuple into a three dimensional tuple
  # without breaking the shape of original array
  P[0, :, :] = np.array([(0, x[0], x[1]) if x else x for x in P_tmp.flatten()]).reshape(P_tmp.shape)
  M[:, 0, : ], P_tmp = scoreTwoSeq(A, C)
  P[:, 0, :]= np.array([(x[0], 0, x[1]) if x else x for x in P_tmp.flatten()]).reshape(P_tmp.shape)
  M[:, :, 0 ], P_tmp = scoreTwoSeq(A, B)
  P[:, :, 0] = np.array([(x[0], x[1], 0) if x else x for x in P_tmp.flatten()]).reshape(P_tmp.shape)
  for a in range(1, len(A) + 1):
    for b in range(1, len(B) + 1):
      for c in range(1, len(C) + 1):
        seven = M[a-1, b-1, c-1] + score[(A[a-1], B[b-1], C[c-1])]
        six   = M[a-1, b-1, c] + score[(A[a-1], B[b-1], '-')]
        five  = M[a-1, b, c-1] + score[(A[a-1], '-', C[c-1])]
        three = M[a-1, b, c] + score[(A[a-1], '-', '-')]
        four  = M[a, b-1, c-1] + score[('-', B[b-1], C[c-1])]
        two   = M[a, b-1, c] + score[('-', B[b-1], '-')]
        one   = M[a, b, c-1] + score[('-', '-', C[c-1])]
        M[a, b, c] =  max([one, two, three, four, five, six, seven])

        if M[a, b, c] == seven:
          P[a, b, c] = (a-1, b-1, c-1)
        elif M[a, b, c] == six:
          P[a, b, c] = (a-1, b-1, c)
        elif M[a, b, c] == four:
          P[a, b, c] = (a, b-1, c-1)
        elif M[a, b, c] == five:
          P[a, b, c] = (a-1, b, c-1)
        elif M[a, b, c] == one:
          P[a, b, c] = (a, b, c-1)
        elif M[a, b, c] == two:
          P[a, b, c] = (a, b-1, c)
        elif M[a, b, c] == three:
          P[a, b, c] = (a-1, b, c)
        else:
          raise ValueError
  #Get the alignment str
  A_aln, B_aln, C_aln = [], [], []
  a, b, c = len(A), len(B), len(C)
  while P[a, b, c]:
    new_a, new_b, new_c = P[a, b, c]
    if a - new_a > 0:
      A_aln.insert(0, A[a-1])
    else:
      A_aln.insert(0, '-')

    if b - new_b > 0:
      B_aln.insert(0, B[b-1])
    else:
      B_aln.insert(0, '-')

    if c - new_c > 0:
      C_aln.insert(0, C[c-1])
    else:
      C_aln.insert(0, '-')
    a, b, c = new_a, new_b, new_c
  max_score = M[len(A), len(B), len(C)]
  return A_aln, B_aln, C_aln, max_score

In [166]:
import numpy as np
import time
import re
from numpy import array

start = time.clock()


A, T, C, G, S = 0, 1, 2, 3, 4
nt = [A,T,C,G,S]
c2i = {'A':0, 'T':1, 'C':2, 'G':3}


# int_to_char = {0:'A', 1:'T', 2:'C', 3:'G', 4:'S'}

match = 5
mismatch = -4
indel = -8

scoring_pair = array([[match,mismatch,mismatch,mismatch,indel],
                 [mismatch,match,mismatch,mismatch,indel],
                 [mismatch,mismatch,match,mismatch,indel],
                 [mismatch,mismatch,mismatch,match,indel],
                 [indel,indel,indel,indel,0]])

# print (scoring_pair)

scoring_three = np.zeros((5,5,5))

for a in nt:
    for b in nt:
        for c in nt:
            scoring_three[a, b, c] = scoring_pair[a,b] + scoring_pair[b,c] + scoring_pair[a,c]
# scoring_three


# print (scoring_three[S,S,S])



def PSA(seq1,seq2):
    global scoring_three
    
    new_match = scoring_three[A,A,S]
    new_mismatch = scoring_three[A,T,S]
    new_indel = scoring_three[A,S,S]

    matrix_table = np.zeros((len(seq1) + 1, len(seq2) + 1))
    trace = np.empty((len(seq1) + 1, len(seq2) + 1), dtype=object)

    for i in range(1, len(seq1) + 1):
        matrix_table[i,0] = i * new_indel
        trace[i,0] = (0, i-1)
    for j in range(1, len(seq2) + 1):
        matrix_table[0,j] = j * new_indel
        trace[0,j] = (j-1, 0)

    
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            one = matrix_table[i-1,j-1] + new_match
            two = matrix_table[i-1,j-1] + new_mismatch
            three = matrix_table[i-1,j] + new_indel
            four = matrix_table[i,j-1] + new_indel
            
            if seq1[i-1] == seq2[j-1]:
                matrix_table[i,j] = max(one, three, four)
            else:
                matrix_table[i,j] = max(two, three, four)
                
            if matrix_table[i,j] == one or matrix_table[i,j] == two:
                trace[i,j] = (i-1,j-1)
            elif matrix_table[i,j] == three:
                trace[i,j] = (i-1,j)
            else:
                trace[i,j] = (i,j-1)

    return (matrix_table, trace)


PSA(seq1,seq2)


seq1 = ''
seq2 = ''
seq3 = ''

with open ('/Users/DONG/Documents/Courses/CS238/hw3/seq/1_556.txt') as f1:
    for line in f1:
        line = re.sub(r"\s+|\d+", "", line).upper()
        seq1 += line

with open ('/Users/DONG/Documents/Courses/CS238/hw3/seq/1_569.txt') as f2:
    for line in f2:
        line = re.sub(r"\s+|\d+", "", line).upper()
        seq2 += line
        
with open ('/Users/DONG/Documents/Courses/CS238/hw3/seq/1_627.txt') as f2:
    for line in f2:
        line = re.sub(r"\s+|\d+", "", line).upper()
        seq3 += line


print (seq1, '\n\n', seq2, '\n\n', seq3)
print (len(seq1))

PSA(seq1,seq2)

s1 = []
s2 = []
s3 = []
for i in range(len(seq1)):
    s1.append(c2i[seq1[i]])
    
for i in range(len(seq2)):
    s2.append(c2i[seq2[i]])
    
for i in range(len(seq3)):
    s3.append(c2i[seq3[i]])
    
print (s1)




#seq1 = 'ATCCG'
#seq2 = 'ATTCG'
#seq3 = 'ATCGG'

D3_table = np.zeros((len(seq1) + 1, len(seq2) + 1, len(seq3) + 1))

D3_trace = np.empty((len(seq1) + 1, len(seq2) + 1, len(seq3) + 1), dtype=object)



D3_table[:, :, 0] = PSA(seq1, seq2)[0]

D3_table[:, 0, :] = PSA(seq1, seq3)[0]

D3_table[0, :, :] = PSA(seq2, seq3)[0]

# print (D3_table)


print (seq1[1])
# print (scoring_three[seq1[0],seq2[1],seq3[3]])

print (D3_table)


for i in range(1, len(s1) + 1):
    for j in range(1, len(s2) + 1):
        for k in range(1, len(s3) + 1):
            one = D3_table[i-1, j-1, k-1] + scoring_three[s1[i-1], s2[j-1], s3[k-1]]
            two   = D3_table[i-1, j-1, k] + scoring_three[s1[i-1], s2[j-1], S]
            three  = D3_table[i-1, j, k-1] + scoring_three[s1[i-1], S, s3[k-1]]
            four = D3_table[i-1, j, k] + scoring_three[s1[i-1],S,S]
            five  = D3_table[i, j-1, k-1] + scoring_three[S, s2[j-1], s3[k-1]]
            six   = D3_table[i, j-1, k] + scoring_three[S, s2[j-1], S]
            seven   = D3_table[i, j, k-1] + scoring_three[S,S,s3[k-1]]
            D3_table[i, j, k] =  max(one, two, three, four, five, six, seven)
            
            
            if D3_table[i, j, k] == one:
                D3_trace[i, j, k] = (i-1, j-1, k-1)
            elif D3_table[i, j, k] == two:
                D3_trace[i, j, k] = (i-1, j-1, k)
            elif D3_table[i, j, k] == three:
                D3_trace[i, j, k] = (i-1, j, k-1)
            elif D3_table[i, j, k] == four:
                D3_trace[i, j, k] = (i-1, j, k)
            elif D3_table[i, j, k] == five:
                D3_trace[i, j, k] = (i, j-1, k-1)
            elif D3_table[i, j, k] == six:
                D3_trace[i, j, k] = (i, j-1, k)
            elif D3_table[i, j, k] == seven:
                D3_trace[i, j, k] = (i, j, k-1)
            
optimal_score = D3_table[i,j,k]          
            

# i2c = {0:'A', 1:'T', 2:'C', 3:'G'}    
    
(align_s1, align_s2, align_s3) = ('', '', '')

i, j, k = len(s1), len(s2), len(s3)


while D3_trace[i, j, k]:
    x, y, z = D3_trace[i, j, k]
    if i > x:
        align_s1 = seq1[i-1] + align_s1
    else:
        align_s1 = '-' + align_s1

    if j > y:
        align_s2 = seq2[j-1] + align_s2
    else:
        align_s2 = '-' + align_s2
    
    if k > z:
        align_s3 = seq3[k-1] + align_s3
    else:
        align_s3 = '-' + align_s3
         
    i, j, k = x, y, z
        
perfect_match = 0
con_seq = ''
for i in range(len(align_s1)):
    if align_s1[i] == align_s2[i] and align_s2[i] == align_s3[i] and align_s3[i] != '-':
        perfect_match +=1;
        con_seq += align_s1[i]
    else:
        con_seq += '*'
        
         
align_length = len(align_s1)        
 
elapsed = (time.clock() - start)
elapsed = round(elapsed,3)
print('Time used:', elapsed, 's')    

print ('Optimal score:', optimal_score)
print ('Alignment length:', align_length)
print ('Perfect matches:', perfect_match)

print ('s1:\n', align_s1, '\n\ns2:\n', align_s2, '\n\ns3:\n', align_s3, '\n\nconsensus sequence:', con_seq)


[[ 5 -4 -4 -4 -8]
 [-4  5 -4 -4 -8]
 [-4 -4  5 -4 -8]
 [-4 -4 -4  5 -8]
 [-8 -8 -8 -8  0]]
4


array([[[ 15.,  -3.,  -3.,  -3., -11.],
        [ -3.,  -3., -12., -12., -20.],
        [ -3., -12.,  -3., -12., -20.],
        [ -3., -12., -12.,  -3., -20.],
        [-11., -20., -20., -20., -16.]],

       [[ -3.,  -3., -12., -12., -20.],
        [ -3.,  15.,  -3.,  -3., -11.],
        [-12.,  -3.,  -3., -12., -20.],
        [-12.,  -3., -12.,  -3., -20.],
        [-20., -11., -20., -20., -16.]],

       [[ -3., -12.,  -3., -12., -20.],
        [-12.,  -3.,  -3., -12., -20.],
        [ -3.,  -3.,  15.,  -3., -11.],
        [-12., -12.,  -3.,  -3., -20.],
        [-20., -20., -11., -20., -16.]],

       [[ -3., -12., -12.,  -3., -20.],
        [-12.,  -3., -12.,  -3., -20.],
        [-12., -12.,  -3.,  -3., -20.],
        [ -3.,  -3.,  -3.,  15., -11.],
        [-20., -20., -20., -11., -16.]],

       [[-11., -20., -20., -20., -16.],
        [-20., -11., -20., -20., -16.],
        [-20., -20., -11., -20., -16.],
        [-20., -20., -20., -11., -16.],
        [-16., -16., -16., -16.,

In [222]:
import time

start = time.clock()

perfect_match = 0
for i in range(len(align_s1)):
    if align_s1[i] == align_s2[i] and align_s2[i] == align_s3[i] and align_s3[i] != '-':
        perfect_match +=1;
        

            
align_length = len(align_s1)        
    
elapsed = (time.clock() - start)
elapsed = round(elapsed,3)
print('Time used:', elapsed, 's')    
    
print ('Optimal score:', optimal_score)
print ('Alignment length:', align_length)
print ('Perfect matches:', perfect_match)

print ('s1:\n', align_s1, '\n\ns2:\n', align_s2, '\n\ns3:\n', align_s3)


Time used: 0.0 s
Optimal score: -6.0
Alignment length: 6
Perfect matches: 2
s1:
 AATCGC 

s2:
 GATTG- 

s3:
 ACTCGC


In [172]:
def scoreThreeSeq(A, B, C): 
    global score
    new_indel = score[A,S,S]
    prev, Path = scoreTwoSeq(B, C)
    new = np.zeros((len(B) + 1, len(C) + 1)) 
    
    for a in range(1, len(A) + 1):
        new[0, 0] = prev[0, 0] + new_indel
        
        # fill in the dumpy row when B == 0
        for c in range(1, len(C) + 1):
            new[0, c] = max(new[0, c-1] + new_indel, prev[0, c] + new_indel, 
                            prev[0, c-1] + score[(A[a-1], C[c-1], '-')])
        
        # fill in the dumpy row when C == 0
        for b in range(1, len(B) + 1):
            new[b, 0] = max(new[b-1, 0] + new_indel, prev[b, 0] + new_indel, 
                            prev[b-1, 0] + score[(A[a-1], B[b-1], '-')])
        
        for b in range(1, len(B) + 1):
            for c in range(1, len(C) + 1):
                
                #1-3 edges, 4-6 diagnoal edges, 7, cube diagnoal edge
                seven = prev[b-1, c-1] + score[(A[a-1], B[b-1], C[c-1])]
                six   = prev[b-1, c] + score[(A[a-1], B[b-1], '-')]
                five  = prev[b, c-1] + score[(A[a-1], '-', C[c-1])]
                three = prev[b, c] + score[(A[a-1], '-', '-')]
                four  = new[b-1, c-1] + score[('-', B[b-1], C[c-1])]
                two   = new[b-1, c] + score[('-', B[b-1], '-')]
                one   = new[b, c-1] + score[('-', '-', C[c-1])]
                new[b, c] =  max([one, two, three, four, five, six, seven])
        prev = copy.deepcopy(new)
    
    return new

array([[    0.,   -16.,   -32., ..., -9072., -9088., -9104.],
       [  -16.,   -20.,   -27., ..., -9067., -9083., -9099.],
       [  -32.,   -36.,   -40., ..., -9062., -9078., -9094.],
       ..., 
       [-8864., -8859., -8854., ..., -6809., -6825., -6841.],
       [-8880., -8875., -8870., ..., -6804., -6820., -6836.],
       [-8896., -8891., -8886., ..., -6808., -6824., -6840.]])

In [ ]:
def paritionBC(M_upper, M_down):
  # see post: http://stackoverflow.com/questions/42519/how-do-you-rotate-a-two-dimensional-array
  reversed_M_down = np.fliplr(np.transpose(np.fliplr(np.transpose(M_down))))
  sum_array = M_upper + reversed_M_down
  index = np.unravel_index(sum_array.argmax(), sum_array.shape)
  return index

def recursive_call(A, B, C):
  if len(A) <= 1 or len(B) <= 1 or len(C) <= 1:
    upper, middle, down, score = alignThreeSeq(A, B, C)
  else:
    xmid = len(A)/2
    matrix_upper = scoreThreeSeq(A[:xmid], B, C)
    matrix_down = scoreThreeSeq(A[xmid:][::-1], B[::-1], C[::-1])
    b, c = paritionBC(matrix_upper, matrix_down)

    upper_l, middle_l, down_l, score_l = recursive_call(A[:xmid], B[:b], C[:c])
    upper_r, middle_r, down_r, score_r = recursive_call(A[xmid:], B[b:], C[c:])

    upper = upper_l + upper_r 
    middle = middle_l + middle_r
    down = down_l + down_r
    score = score_l + score_r

  return upper, middle, down, score

In [59]:
seq1 = 'ATCG'
seq2 = 'ATTCG'

T = np.zeros((len(seq1) + 1, len(seq2) + 1))
P = np.empty((len(seq1) + 1, len(seq2) + 1), dtype=object)
T[1,2]

0.0

In [ ]:
for i in range(1, len(seq1) + 1):
    for j in range(1, len(seq2) + 1):
        for k in range(1, len(seq3) + 1):
            one = D3_table[i-1, j-1, k-1] + scoring_three[seq1[i-1], seq2[j-1], seq3[k-1]]
            two   = D3_table[i-1, j-1, k] + scoring_three[seq1[i-1], seq2[j-1], S]
            three  = D3_table[i-1, j, k-1] + scoring_three[seq1[i-1], S, seq3[k-1]]
            four = D3_table[i-1, j, k] + scoring_three[seq1[i-1],S,S]
            five  = D3_table[i, j-1, k-1] + scoring_three[S, seq2[j-1], seq3[k-1]]
            six   = D3_table[i, j-1, k] + scoring_three[S, seq2[j-1], S]
            seven   = D3_table[i, j, k-1] + scoring_three[S,S,seq3[k-1]]
            D3_table[i, j, k] =  max(one, two, three, four, five, six, seven)

        if M[a, b, c] == seven:
          P[a, b, c] = (a-1, b-1, c-1)
        elif M[a, b, c] == six:
          P[a, b, c] = (a-1, b-1, c)
        elif M[a, b, c] == four:
          P[a, b, c] = (a, b-1, c-1)
        elif M[a, b, c] == five:
          P[a, b, c] = (a-1, b, c-1)
        elif M[a, b, c] == one:
          P[a, b, c] = (a, b, c-1)
        elif M[a, b, c] == two:
          P[a, b, c] = (a, b-1, c)
        elif M[a, b, c] == three:
          P[a, b, c] = (a-1, b, c)
        else:
          raise ValueError

In [175]:
###### read sequence

import re

seq1 = ''
seq2 = ''
seq3 = ''

with open ('/Users/DONG/Documents/Courses/CS238/hw3/seq/1_556.txt') as f1:
    for line in f1:
        line = re.sub(r"\s+|\d+", "", line).upper()
        seq1 += line

with open ('/Users/DONG/Documents/Courses/CS238/hw3/seq/1_569.txt') as f2:
    for line in f2:
        line = re.sub(r"\s+|\d+", "", line).upper()
        seq2 += line
        
with open ('/Users/DONG/Documents/Courses/CS238/hw3/seq/1_627.txt') as f2:
    for line in f2:
        line = re.sub(r"\s+|\d+", "", line).upper()
        seq3 += line


print (seq1, '\n\n', seq2, '\n\n', seq3)
print (len(seq1), len(s2), len(s3))


c2i = {'A':0, 'T':1, 'C':2, 'G':3}
s1 = []
s2 = []
s3 = []
for i in range(len(seq1)):
    s1.append(c2i[seq1[i]])
    
for i in range(len(seq2)):
    s2.append(c2i[seq2[i]])
    
for i in range(len(seq3)):
    s3.append(c2i[seq3[i]])
    
print (len(s1), len(s2), len(s3))



ACATTCTCCTTCTGATAGACTCAGGAAGCAATCATGGTGCTCTCTGCAGATGACAAAACCAACATCAAGAACTGCTGGGGGAAGATTGGTGGCCATGGTGGTGAATATGGCGAGGAGGCCCTACAGAGGATGTTCGCTGCCTTCCCCACCACCAAGACCTACTTCTCTCACATTGATGTAAGCCCCGGCTCTGCCCAGGTCAAGGCTCACGGCAAGAAGGTTGCTGATGCCTTGGCCAAAGCTGCAGACCACGTCGAAGACCTGCCTGGTGCCCTGTCCACTCTGAGCGACCTGCATGCCCACAAACTGCGTGTGGATCCTGTCAACTTCAAGTTCCTGAGCCACTGCCTGCTGGTGACCTTGGCTTGCCACCACCCTGGAGATTTCACACCCGCCATGCACGCCTCTCTGGACAAATTCCTTGCCTCTGTGAGCACTGTGCTGACCTCCAAGTACCGTTAAGCCGCCTCCTGCCGGGCTTGCCTTCTGACCAGGCCCTTCTTCCCTCCCTTGCACCTATACCTCTTGGTCTTTGAATAAAGCCTGAGTAGGAAGC 

 GACACTTCTGATTCTGACAGACTCAGGAAGAAACCATGGTGCTCTCTGGGGAAGACAAAAGCAACATCAAGGCTGCCTGGGGGAAGATTGGTGGCCATGGTGCTGAATATGGAGCTGAAGCCCTGGAAAGGATGTTTGCTAGCTTCCCCACCACCAAGACCTACTTCCCTCACTTTGATGTAAGCCACGGCTCTGCCCAGGTCAAGGGTCACGGCAAGAAGGTCGCCGATGCTCTGGCCAATGCTGCAGGCCACCTCGATGACCTGCCCGGTGCCCTGTCTGCTCTGAGCGACCTGCATGCCCACAAGCTGCGTGTGGATCCCGTCAACTTCAAGCTCCTGAGCCACTGCCTGCTGGTGACCTTGGCTAGCCACCACCCTGCCGATTTCACCCCCGCGGTGCATGCCTCTCTGGACAAATTCCTTGCCTCTGTGAGCACC

In [176]:
#### initialization

D3_table = np.zeros((len(seq1) + 1, len(seq2) + 1, len(seq3) + 1))

D3_trace = np.empty((len(seq1) + 1, len(seq2) + 1, len(seq3) + 1), dtype=object)



D3_table[:, :, 0] = PSA(seq1, seq2)[0]

D3_table[:, 0, :] = PSA(seq1, seq3)[0]

D3_table[0, :, :] = PSA(seq2, seq3)[0]

print (D3_table)


[[[     0.    -16.    -32. ..., -10000. -10016. -10032.]
  [   -16.    -20.    -36. ...,  -9995. -10011. -10027.]
  [   -32.    -36.    -31. ...,  -9990. -10006. -10022.]
  ..., 
  [ -9072.  -9067.  -9062. ...,  -8092.  -8108.  -8124.]
  [ -9088.  -9083.  -9078. ...,  -8087.  -8103.  -8119.]
  [ -9104.  -9099.  -9094. ...,  -8082.  -8098.  -8114.]]

 [[   -16.    -20.    -27. ...,  -9995. -10011. -10027.]
  [   -20.      0.      0. ...,      0.      0.      0.]
  [   -27.      0.      0. ...,      0.      0.      0.]
  ..., 
  [ -9067.      0.      0. ...,      0.      0.      0.]
  [ -9083.      0.      0. ...,      0.      0.      0.]
  [ -9099.      0.      0. ...,      0.      0.      0.]]

 [[   -32.    -27.    -40. ...,  -9990. -10006. -10022.]
  [   -36.      0.      0. ...,      0.      0.      0.]
  [   -40.      0.      0. ...,      0.      0.      0.]
  ..., 
  [ -9062.      0.      0. ...,      0.      0.      0.]
  [ -9078.      0.      0. ...,      0.      0.      0.]
  [

In [ ]:
### recurrence

for i in range(1, len(s1) + 1):
    for j in range(1, len(s2) + 1):
        for k in range(1, len(s3) + 1):
            one = D3_table[i-1, j-1, k-1] + scoring_three[s1[i-1], s2[j-1], s3[k-1]]
            two   = D3_table[i-1, j-1, k] + scoring_three[s1[i-1], s2[j-1], S]
            three  = D3_table[i-1, j, k-1] + scoring_three[s1[i-1], S, s3[k-1]]
            four = D3_table[i-1, j, k] + scoring_three[s1[i-1],S,S]
            five  = D3_table[i, j-1, k-1] + scoring_three[S, s2[j-1], s3[k-1]]
            six   = D3_table[i, j-1, k] + scoring_three[S, s2[j-1], S]
            seven   = D3_table[i, j, k-1] + scoring_three[S,S,s3[k-1]]
            D3_table[i, j, k] =  max(one, two, three, four, five, six, seven)

print (D3_table[len(s1),len(s2),len(s3)])

In [180]:
##### scoring matrix

match = 5
mismatch = -4
indel = -8

scoring_pair = array([[match,mismatch,mismatch,mismatch,indel],
                 [mismatch,match,mismatch,mismatch,indel],
                 [mismatch,mismatch,match,mismatch,indel],
                 [mismatch,mismatch,mismatch,match,indel],
                 [indel,indel,indel,indel,0]])

# print (scoring_pair)

scoring_three = np.zeros((5,5,5))

for a in nt:
    for b in nt:
        for c in nt:
            scoring_three[a, b, c] = scoring_pair[a,b] + scoring_pair[b,c] + scoring_pair[a,c]
scoring_three

array([[[ 15.,  -3.,  -3.,  -3., -11.],
        [ -3.,  -3., -12., -12., -20.],
        [ -3., -12.,  -3., -12., -20.],
        [ -3., -12., -12.,  -3., -20.],
        [-11., -20., -20., -20., -16.]],

       [[ -3.,  -3., -12., -12., -20.],
        [ -3.,  15.,  -3.,  -3., -11.],
        [-12.,  -3.,  -3., -12., -20.],
        [-12.,  -3., -12.,  -3., -20.],
        [-20., -11., -20., -20., -16.]],

       [[ -3., -12.,  -3., -12., -20.],
        [-12.,  -3.,  -3., -12., -20.],
        [ -3.,  -3.,  15.,  -3., -11.],
        [-12., -12.,  -3.,  -3., -20.],
        [-20., -20., -11., -20., -16.]],

       [[ -3., -12., -12.,  -3., -20.],
        [-12.,  -3., -12.,  -3., -20.],
        [-12., -12.,  -3.,  -3., -20.],
        [ -3.,  -3.,  -3.,  15., -11.],
        [-20., -20., -20., -11., -16.]],

       [[-11., -20., -20., -20., -16.],
        [-20., -11., -20., -20., -16.],
        [-20., -20., -11., -20., -16.],
        [-20., -20., -20., -11., -16.],
        [-16., -16., -16., -16.,

In [198]:
### get sequence

seq1 = 'AATCGC'
seq2 = 'GGATTG'
seq3 = 'ACTCGC'

c2i = {'A':0, 'T':1, 'C':2, 'G':3}
s1 = []
s2 = []
s3 = []
for i in range(len(seq1)):
    s1.append(c2i[seq1[i]])
    
for i in range(len(seq2)):
    s2.append(c2i[seq2[i]])
    
for i in range(len(seq3)):
    s3.append(c2i[seq3[i]])
    
print (len(s1), len(s2), len(s3))



6 6 6


In [199]:
#### initialization

D3_table = np.zeros((len(seq1) + 1, len(seq2) + 1, len(seq3) + 1))

D3_trace = np.empty((len(seq1) + 1, len(seq2) + 1, len(seq3) + 1), dtype=object)



D3_table[:, :, 0] = PSA(seq1, seq2)[0]

D3_table[:, 0, :] = PSA(seq1, seq3)[0]

D3_table[0, :, :] = PSA(seq2, seq3)[0]

print (D3_table)
print (D3_trace)

[[[   0.  -16.  -32.  -48.  -64.  -80.  -96.]
  [ -16.  -20.  -36.  -52.  -68.  -75.  -91.]
  [ -32.  -36.  -40.  -56.  -72.  -79.  -95.]
  [ -48.  -43.  -56.  -60.  -76.  -92.  -99.]
  [ -64.  -59.  -63.  -67.  -80.  -96. -112.]
  [ -80.  -75.  -79.  -74.  -87. -100. -116.]
  [ -96.  -91.  -95.  -90.  -94.  -98. -114.]]

 [[ -16.  -11.  -27.  -43.  -59.  -75.  -91.]
  [ -20.    0.    0.    0.    0.    0.    0.]
  [ -36.    0.    0.    0.    0.    0.    0.]
  [ -43.    0.    0.    0.    0.    0.    0.]
  [ -59.    0.    0.    0.    0.    0.    0.]
  [ -75.    0.    0.    0.    0.    0.    0.]
  [ -91.    0.    0.    0.    0.    0.    0.]]

 [[ -32.  -27.  -31.  -47.  -63.  -79.  -95.]
  [ -36.    0.    0.    0.    0.    0.    0.]
  [ -40.    0.    0.    0.    0.    0.    0.]
  [ -47.    0.    0.    0.    0.    0.    0.]
  [ -63.    0.    0.    0.    0.    0.    0.]
  [ -79.    0.    0.    0.    0.    0.    0.]
  [ -95.    0.    0.    0.    0.    0.    0.]]

 [[ -48.  -43.  -47.  -42.  

In [204]:
#### 

for i in range(1, len(s1) + 1):
    for j in range(1, len(s2) + 1):
        for k in range(1, len(s3) + 1):
            one = D3_table[i-1, j-1, k-1] + scoring_three[s1[i-1], s2[j-1], s3[k-1]]
            two   = D3_table[i-1, j-1, k] + scoring_three[s1[i-1], s2[j-1], S]
            three  = D3_table[i-1, j, k-1] + scoring_three[s1[i-1], S, s3[k-1]]
            four = D3_table[i-1, j, k] + scoring_three[s1[i-1],S,S]
            five  = D3_table[i, j-1, k-1] + scoring_three[S, s2[j-1], s3[k-1]]
            six   = D3_table[i, j-1, k] + scoring_three[S, s2[j-1], S]
            seven   = D3_table[i, j, k-1] + scoring_three[S,S,s3[k-1]]
            D3_table[i, j, k] =  max(one, two, three, four, five, six, seven)
            
            
            if D3_table[i, j, k] == one:
                D3_trace[i, j, k] = (i-1, j-1, k-1)
            elif D3_table[i, j, k] == two:
                D3_trace[i, j, k] = (i-1, j-1, k)
            elif D3_table[i, j, k] == three:
                D3_trace[i, j, k] = (i-1, j, k-1)
            elif D3_table[i, j, k] == four:
                D3_trace[i, j, k] = (i-1, j, k)
            elif D3_table[i, j, k] == five:
                D3_trace[i, j, k] = (i, j-1, k-1)
            elif D3_table[i, j, k] == six:
                D3_trace[i, j, k] = (i, j-1, k)
            elif D3_table[i, j, k] == seven:
                D3_trace[i, j, k] = (i, j, k-1)
            
optimal_score = D3_table[i,j,k]          
            

# i2c = {0:'A', 1:'T', 2:'C', 3:'G'}    
    
(align_s1, align_s2, align_s3) = ('', '', '')

i, j, k = len(s1), len(s2), len(s3)


while D3_trace[i, j, k]:
    x, y, z = D3_trace[i, j, k]
    if i > x:
        align_s1 = seq1[i-1] + align_s1
    else:
        align_s1 = '-' + align_s1

    if j > y:
        align_s2 = seq2[j-1] + align_s2
    else:
        align_s2 = '-' + align_s2
    
    if k > z:
        align_s3 = seq3[k-1] + align_s3
    else:
        align_s3 = '-' + align_s3
         
    i, j, k = x, y, z
        

align_length = len(align_s1)        
        
print ('Optimal score:', optimal_score)
print ('Alignment length:', align_length)

print ('s1:\n', align_s1, '\n\ns2:\n', align_s2, '\n\ns3:\n', align_s3)



Optimal score: -6.0
Alignment length: 6
s1:
 AATCGC 

s2:
 GATTG- 

s3:
 ACTCGC


In [188]:
(align_s1, align_s2, align_s3) = ('', '', '')
print (align_s1)

In [190]:
mmm = 1
print (chr(mmm))




In [221]:
import time

start = time.clock()


elapsed = (time.clock() - start)
elapsed = round(elapsed,3)
print('Time used:', elapsed, 's')

108000001
Time used: 17.101 s
